In [1]:
from sklearn.datasets import load_iris

In [2]:
iris_data = load_iris()

## (1) 학습/테스트 데이터 셋 분리하지 않고 예측

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier()

train_data = iris_data.data
train_label = iris_data.target

# 학습
dt_clf.fit(train_data, train_label)

# 테스트
pred = dt_clf.predict(train_data)
print(f'예측 정확도 : {accuracy_score(train_label, pred)}')

예측 정확도 : 1.0


- 분리하지 않아서 정확도가 1.0이 나옴 (잘못된 결과)

## (2) 분리

In [10]:
from sklearn.model_selection import train_test_split

# 분리
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label,
                                                   test_size=0.3, random_state=4)

dt_clf = DecisionTreeClassifier()

dt_clf.fit(X_train, y_train)

pred = dt_clf.predict(X_test)
print(f'예측 정확도 : {accuracy_score(y_test, pred)}')

예측 정확도 : 0.9777777777777777


### Pandas DataFrame/Series도 가능

In [9]:
import pandas as pd

iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['target'] = iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [11]:
feature_df = iris_df.iloc[:, :-1]
label_df = iris_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df,
                                                   test_size=0.3, random_state=4)

dt_clf = DecisionTreeClassifier()

dt_clf.fit(X_train, y_train)

pred = dt_clf.predict(X_test)
print(f'예측 정확도 : {accuracy_score(y_test, pred)}')

예측 정확도 : 0.9777777777777777


In [12]:
X_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
13,4.3,3.0,1.1,0.1
15,5.7,4.4,1.5,0.4
61,5.9,3.0,4.2,1.5
91,6.1,3.0,4.6,1.4
116,6.5,3.0,5.5,1.8


# Data Split과 모델 검증
## 1. 홀드아웃
- 데이터를 Train, Test 세트로 나누어 사용
- 과적합
    - 모델이 학습 데이터에 과도하게 최적화되어 다른 테스트 데이터로 예측할 경우 성능이 떨어지는 것
    - Train, Test로만 나눠놓으면 test에 맞는 파라미터를 맞추기 때문에 과적합 발생

## 2. 교차검증
- k-fold Cross Validation
- training 데이터를 임의의 k개 그룹으로 나눈 뒤 그 중 하나를 test(validation), 나머지 k-1개를 train으로 k번 반복하여 나온 평가를 평균

In [14]:
from sklearn.model_selection import KFold

print(f'붓꽃 데이터 세트 크기 : {len(iris_data.data)}')

붓꽃 데이터 세트 크기 : 150


In [18]:
features = iris_data.data
label = iris_data.target

In [19]:
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리
kfold = KFold(n_splits=5)

# 폴드 세트별 정확도 담을 리스트
cv_accuracy = []

In [23]:
# 폴드별 학습용, 검증용 데이터 세트의 행 인덱스
for train_index, test_index in kfold.split(features):
    print(f'학습용 : {train_index}')
    print(f'검증용 : {test_index}')
    print()

학습용 : [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
검증용 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

학습용 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 12

In [24]:
# K-fold 검증
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    
    pred = dt_clf.predict(X_test)
    acc = accuracy_score(pred, y_test)
    cv_accuracy.append(acc)
    
    train_size, test_size = X_train.shape[0], X_test.shape[0]
    print(f'정확도 : {acc:.3f}, 학습 데이터 크기 : {train_size}, 검증 데이터 크기 : {test_size}')
    print(f'검증 세트 인덱스 : {test_index}')
    print()

정확도 : 1.000, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

정확도 : 0.967, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

정확도 : 0.867, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

정확도 : 0.933, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

정확도 : 0.733, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]



In [25]:
mean = sum(cv_accuracy)/len(cv_accuracy)
print(f'평균 검증 정확도 : {mean:.3f}')

평균 검증 정확도 : 0.900


## Stratified K-Fold
- 불량률이 낮은 생산라인에서 양품, 불량품 예측 문제나 의료 문제 등
- 불균형한 데이터 문제를 해결하는 방법 중 하나
    - 데이터 추가 확보
    - Re-Sampling
        - Under-sampling(과소표집)
           - 다른 데이터에 비해 많은 데이터를 줄임
           - 유용한 정보 손실이 있을 수 있음
        - Over-sampling(과대표집)
            - 적은 데이터를 복제하여 데이터를 늘림
            - 과적합 위험

In [26]:
iris_df['target'].value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

In [32]:
# 3개 폴드 구성
kfold = KFold(n_splits=3)

n = 0
for train_index, test_index in kfold.split(iris_df):
    n += 1
    
    label_train = iris_df['target'].iloc[train_index]
    label_test = iris_df['target'].iloc[test_index]
    
    
    print("## 교차검증 :", n)
    print("학습용 :", label_train.value_counts())
    print("검증용 :", label_test.value_counts())
    print()

## 교차검증 : 1
학습용 : 1    50
2    50
Name: target, dtype: int64
검증용 : 0    50
Name: target, dtype: int64

## 교차검증 : 2
학습용 : 0    50
2    50
Name: target, dtype: int64
검증용 : 1    50
Name: target, dtype: int64

## 교차검증 : 3
학습용 : 0    50
1    50
Name: target, dtype: int64
검증용 : 2    50
Name: target, dtype: int64



이런 경우 매우 불량한 예측 정확도가 나온다.

In [33]:
# 3개 폴드 구성
kfold = KFold(n_splits=3)
cv_accuracy = []
n = 0
for train_index, test_index in kfold.split(iris_df):
    n += 1
    
    X_train = iris_df.iloc[train_index, :-1]
    X_test = iris_df.iloc[test_index, :-1]
    y_train = iris_df.iloc[train_index, -1]
    y_test = iris_df.iloc[test_index, -1]
    
    dt_clf.fit(X_train, y_train)
    
    pred = dt_clf.predict(X_test)
    acc = accuracy_score(pred, y_test)
    cv_accuracy.append(acc)
    
    train_size, test_size = X_train.shape[0], X_test.shape[0]
    print("## 교차검증 :", n)
    print(f'정확도 : {acc:.3f}, 학습 데이터 크기 : {train_size}, 검증 데이터 크기 : {test_size}')
    print(f'검증 세트 인덱스 : {test_index}')
    print()

mean = sum(cv_accuracy)/len(cv_accuracy)
print(f'평균 검증 정확도 : {mean:.3f}')

## 교차검증 : 1
정확도 : 0.000, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]

## 교차검증 : 2
정확도 : 0.000, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
검증 세트 인덱스 : [50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]

## 교차검증 : 3
정확도 : 0.000, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
검증 세트 인덱스 : [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

평균 검증 정확도 : 0.000


### StratifiedKFold
- KFold와 다른 점 : 레이블 데이터 세트 인수가 필요함
    - 레이블 데이터 세트로 Stratify 하기 때문

In [35]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n = 0
for train_index, test_index in skf.split(iris_df, iris_df['target']):
    n += 1
    
    label_train = iris_df['target'].iloc[train_index]
    label_test = iris_df['target'].iloc[test_index]
    
    
    print("## 교차검증 :", n)
    print("학습용 :", label_train.value_counts())
    print("검증용 :", label_test.value_counts())
    print()

## 교차검증 : 1
학습용 : 2    34
0    33
1    33
Name: target, dtype: int64
검증용 : 0    17
1    17
2    16
Name: target, dtype: int64

## 교차검증 : 2
학습용 : 1    34
0    33
2    33
Name: target, dtype: int64
검증용 : 0    17
2    17
1    16
Name: target, dtype: int64

## 교차검증 : 3
학습용 : 0    34
1    33
2    33
Name: target, dtype: int64
검증용 : 1    17
2    17
0    16
Name: target, dtype: int64



In [36]:
# 3개 폴드 구성
cv_accuracy = []
skf = StratifiedKFold(n_splits=3)
n = 0
for train_index, test_index in skf.split(iris_df, iris_df['target']):
    n += 1
    
    X_train = iris_df.iloc[train_index, :-1]
    X_test = iris_df.iloc[test_index, :-1]
    y_train = iris_df.iloc[train_index, -1]
    y_test = iris_df.iloc[test_index, -1]
    
    dt_clf.fit(X_train, y_train)
    
    pred = dt_clf.predict(X_test)
    acc = accuracy_score(pred, y_test)
    cv_accuracy.append(acc)
    
    train_size, test_size = X_train.shape[0], X_test.shape[0]
    print("## 교차검증 :", n)
    print(f'정확도 : {acc:.3f}, 학습 데이터 크기 : {train_size}, 검증 데이터 크기 : {test_size}')
    print(f'검증 세트 인덱스 : {test_index}')
    print()

mean = sum(cv_accuracy)/len(cv_accuracy)
print(f'평균 검증 정확도 : {mean:.3f}')

## 교차검증 : 1
정확도 : 0.980, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

## 교차검증 : 2
정확도 : 0.940, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

## 교차검증 : 3
정확도 : 0.980, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

평균 검증 정확도 : 0.967


회귀에서는 지원되지 않음

### for문을 이용하지 않고 간단하게 확인
### cross_val_score(estimator, X, y, scoring, cv, ...)
- estimator : Classifier or Regressor
    - 분류 모델이면 **Stratified KFold**, 회귀 모델이면 일반적인 KFold
- X : feature
- y : label
- scoring : 예측 성능 평가 지표 (정확도, 정밀도, 재현율 등)
- cv : 교차검증 폴드 수
- 반환값 : 성능 지표 측정값을 담은 array

In [43]:
from sklearn.model_selection import cross_val_score, cross_validate
import numpy as np

dt_clf = DecisionTreeClassifier(random_state=156)
arr = cross_val_score(dt_clf, features, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도 :', arr)
print('평균 검증 정확도 :', np.round(arr.mean(), 3))

교차 검증별 정확도 : [0.98 0.94 0.98]
평균 검증 정확도 : 0.967


## 하이퍼 파라미터 튜닝까지
- 알고리즘 구성 요소
- 최고 성능 파라미터 찾기

### GridSearchCV
- Grid처럼 모든 조합을 다 테스트 해보는 것
- estimator : classifier, regressor, peipeline
    - `cross_val_score`처럼 분류 모델이면 **Stratified KFold** 이용
- param_grid : {key:list} dictionary
    - key : 파라미터 명
    - list : 파라미터 값
- scoring : 예측 성능 평가 지표
- cv : 교차검증 폴드 수
- refit : 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습 여부

In [45]:
from sklearn.model_selection import GridSearchCV

# GirdSearchCV를 이용하기 위해서는 split 되어있어야 함
X_train, X_test, y_train, y_test = train_test_split(features, label,
                                                    test_size=0.2, random_state=121)

dt_clf = DecisionTreeClassifier()

# DecisionTreeClassifier()의 파라미터 생성
# max_depth : Tree의 최대 깊이
# min_samples_split : 자식 노드를 만드는데 필요한 최소 샘플 개수
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2, 3]}

In [46]:
grid_tree = GridSearchCV(dt_clf, param_grid = parameters, cv=3, refit=True,
                 return_train_score=True) # 최상의 학습 결과 return
grid_tree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_tree.cv_results_) # .cv_results_에 결과가 저장되어 있음

In [47]:
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000997,1.123916e-07,0.000000,0.00000,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
1,0.000332,4.701340e-04,0.000333,0.00047,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
2,0.000665,4.701341e-04,0.000332,0.00047,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
3,0.000332,4.700217e-04,0.000333,0.00047,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
4,0.000332,4.701340e-04,0.000000,0.00000,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02
5,0.000332,4.701340e-04,0.000333,0.00047,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02


In [48]:
scores_df[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1


In [49]:
grid_tree.best_params_

{'max_depth': 3, 'min_samples_split': 2}

In [50]:
grid_tree.best_score_

0.975

In [52]:
# refit=True 이므로 새로 재학습시킬 필요가 없이 바로 훈련된 모델 사용 가능
best_dt = grid_tree.best_estimator_
pred = grid_tree.predict(X_test)
print(f'예측 정확도 : {accuracy_score(y_test, pred)}')

예측 정확도 : 0.9666666666666667
